# Basic Neural Network 

without using tensorflow

### Operations

In [96]:
class Operation:
    
    def __init__(self, input_nodes):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in self.input_nodes:
            node.output_nodes.append(self)
            
        _def_graph.operations.append(self)
            
    def compute(self):
        pass

In [97]:
class Add(Operation):
    
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x + y

In [98]:
class Multiply(Operation):
    
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x * y

In [99]:
class MatMul(Operation):
        
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x, y):
        self.inputs = [x, y]
        return x.dot(y)

### Placeholder

In [100]:
class Placeholder:
    
    def __init__(self):
        self.output_nodes = []
        
        _def_graph.placeholders.append(self)

### Variable

In [101]:
class Variable:
    
    def __init__(self, initial_value=None):
        self.value = initial_value
        self.output_nodes = []
        
        _def_graph.variables.append(self)

### Graph

In [102]:
class Graph:
    
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_default(self):
        global _def_graph
        _def_graph = self

### Example  (`z = Ax + b`)

z = 10x + 1

In [103]:
g = Graph()
g.set_default()

A = Variable(10)
b = Variable(1)
x = Placeholder()
y = Multiply(A, x)
z = Add(y, b)

### Travese operation to build the evaluation order

In [104]:
def traverse_postorder(operation):
    nodes_postorder = []
    
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
        
    recurse(operation)
    return nodes_postorder

### Session

In [105]:
import numpy as np

class Session:
    
    def run(self, operation, feed_dict={}):
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            if type(node) == Placeholder:
                node.output = feed_dict[node]
            elif type(node) == Variable:
                node.output = node.value
            else:
                # Operation
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)
            
            if type(node.output) == list:
                node.output = np.array(node.output)
            
        return operation.output

### Execute 

In [106]:
sess = Session()
result = sess.run(operation=z, feed_dict={x:10})

result

101

### Matrix multiplication example

In [107]:
g = Graph()
g.set_default()

A = Variable([[10,20],[30,40]])
b = Variable([1,2])

x = Placeholder()
y = MatMul(A, x)

z = Add(y, b)

In [108]:
sess = Session()
sess.run(operation=z, feed_dict={x:10})

array([[101, 202],
       [301, 402]])